# Add Python Source Code Hot Reload

## Overview

The application is currently able to visualize kvlang files, providing real-time updates each time the user updates a kvlang script. 

This issue is meant to expand the hot-reload visualization capabilities to include the visualization of kivy python types. This update should allow the user to visualize any custom defined kv widgets or kv application classes. The visualized code should run separate from the `kivydesigner` environment, and should use the end-user's development environment to execute the visualized code.

# Research 

## How can we execute code using a completely different version of kivy?

To run a different version of kivy we would need to configure the visualization subprocess kernel to point towards a python environment with a different version of kivy installed. This is possible to achieve since we are purposefully visualizing the code in a separate process, but this will take some work and research. 

To simpify this current issue, let's visualize all of the end-user code using the same version of kivy as `kivydesigner`. The current design should be flexible enough to allow this future update. 

## How can we execute an end-user kivy class or widget in a subprocess we manage?

### Requirement 1 of 3: Isolate our app environment from end-user's environment 
We need to ensure a clear separation between the `kivydesigner` process environment and the child process visualization environment. We should avoid sharing any environmental resources across the parent and child process. I believe this is possible using the `multiprocessing` module if we specify the `spawn` startup method. The `spawn`ed process will only share the minimum amount of resources required to execute the `run` method. As long as our run method only contains standard python objects (`str` of picked objects) then we should be able to entirely separate the environments. 

### Requirement 2 of 3: Configure end-user kivy environment to recognize custom widgets
We will need to configure the kivy environment within the child process to match the requirements of the end-user application. This means that we need to update the child process kivy `Factory` to register each of the end-user's custom widgets, and the `Builder` to include each of the end-user's builder rules & styles. To start we can take the naive approach and pass a full list of kv files & widgets to the child process & direct the child process to register each item.  

### Requirement 3 of 3: Restore visualization environment to clean state between reloads
We need to design the visualization subprocess to be able to hot reload the code. This means we should be able to restore the environment to a clean state, and re-import each of the impacted modules. Hot reloading code in a limited way is hard to achieve without some specific knowledge of the code, because modules can have a complex interdependency. It *might* be possible to start with a clean environment and build an import tree. We might be able to use such an import tree to selectively reload portion of the code by trimming & reloading each of the child nodes of an impacted module. That would require a whole bunch of research, and should be left as a future possible refinement. For now we can take the brute force approach of deleteing all instance of the end-user's application and forcibly hot reloading everything.

The absolute simpliest way to achieve this would be to restart the visualization interpreter session. This would have two major downsides: 1) restarting an interpreter is slow, and would cause a large delay between reload updates, 2) restarting the intepreter would close the current visualization window. It is much more user friendly to maintain a single visualization window through the hot reload development process. 

To avoid those two major downsides, we will maintain the current visualization approach. This appoach maintains a single window instance, and explicitely does not reload the kivy module between reloads. 

## How does the kivy designer visualize end user apps?

The `kivydesigner` starts a separate process within `KivyDesignerApp._start_visualizing`. The process is instructed to execute the `hotreload.run_visualization_app` method. This method establishes an execution loop for the newly created process. The execution loop is controlled by the `visualization_instruction` instruction queue. Each time an instruction is received (unless it is a STOP instruction), a new kivy `App` is stopped and restared using the new instruction. The `App` is restarted in a way that maintains a single instance of the `kivy` `window` class. Since the visualization subprocess only supports `kvlang` file updates, the visualization process currently only creates and restarts a custom kvlang viewer app class `KvBuilderApp`. In principal, however, any app type can be visualized here. 

We will need to add instructions that allow visualizing custom applications and custom widgets.

```
# kivydesigner.py
def _start_visualizing(self):
        '''
        Run the VisualizationSubprocess on a new child process. 
        Hot reloads can be performed by putting reload instructions
        onto the visualization_instructions queue. 
        '''
        # We are taking special care to avoid creating VisualizationSubprocess within
        # this interpreter session to avoid initializing the VisualizationSubprocess 
        # app using the KivyDesignerApp config. Kivy's initialization relies on global 
        # singletons, so mixing the environments will cause the visualization to fail.
        new_process = multiprocessing.Process(
            target=run_visualization_app,
            args=(self.visualization_instructions,)
        )
        new_process.start()
        self.visualization_subprocess = new_process
```

## What is the sys.modules dictionary?

The `sys.modules` dictionary that maps imported module names to their corresponding module objects. Each time a module is imported python adds an entry to this dictionary. 

The `sys.modules` dictionary can be used to check if a module has been imported, as follows:

```
if 'my_module' in sys.modules:
    # my_module has been imported
    my_module = sys.modules['my_module']
else:
    # my_module has not been imported
    import my_module
```

This dictionary is read-only. 

## Can you remove entries from sys.modules?

You can delete module entries from the `sys.modules` dictionary by calling `del sys.modules['my_module']`, but this command will not actually unload the module or remove it from memory. 

## How do you hot reload a python module?

The python `importlib` provides a function to perform hot reloads on a module `importlib.reload(module_name: str)`. 

This method requires that the module have been successfully imported before. This method forces the python module code to be recompiled, and re-executes the module-level code. This method does not, however, does not delete references to the preexisting objects that were instantiated with the old definition. The reload function also retains the sys.module dictionary for that module. This is generally not a problem between the existing entries are typically overwritten during a reload - but this will cause old definitions to remain if the module was updated to delete or rename an item. 

## Can we forcefully unload an imported python module?

It is possible to directy python to directly unload a module. Doing so requires a careful handling of module references and understanding of the garbage collection system. 

The `gc` module contains a method `gc.collect` that triggers the garbage collector to release any unreferenced objects, including modules that have been removed from the sys.modules dictionary. The garbage collector will only unload the module, however, if all references to objects defined within that module have been released. Therefore, if attempting to use gc.collect to forcefully unload the module we need to 1) carefully dereference all of the objects contained within that module, and 2) remove the modules from the `sys.modules` dictionary. 

There are major downsides to this approach, however:
1) `gc.collect` is an expensive function call. This method will scan the heap to search for all objects with ref count of zero, so there could be a meaningful delay between reloads (of course, we should test this if the gc.collect approach has enough advantages)
2) The `gc.collect` standard documentation does not contain any information about using the garbage collector to unload a module. In general all unreferenced objects should be destroyed when there are no outstanding references. I'm assuming that this would apply to module objects as well - but there very well could be some exceptions made for modules that I'm not aware of
3) I cannot think of a way to explicitely test if a module other than testing if the module is stored within the `sys.modules` dictionary. This is not a perfect test though because deleting an entry from this dictionary does not unload the module. 

Overall I think it is just a lot safer to use the `importlib` library

# General Solution Plan 

1. Update the visualization process creation code to explicitely specify the `spawn` startup mode. Other operating systems default to other startup modes, and these modes may share more resources in order to speed up the interpreter spinup. 
2. When the visualization subprocess is first created, import the kivy module & libs need by visualizer and then cache the following information:
    * What modules are already available within sys.modules?
    * Grab the initial state of the kivy `Factory` and `Builder`
      * I'm not sure how to do this. Worst case we could pickle the initial objects and then directly overwrite them. It might be more efficient to grab the list of names/rules and explictely reset the managers. We'll see
3. When the visualized application is stopped for a reload, perform the following environment cleanup tasks:
    * Remove all references to existing visualized objects. 
      * Set the stopped app to None 
      * Unload all non-cached items from the `kivy` `Factory` & `Builder`
    * Use the python `importlib.reload` function to reload every modules in `sys.module` that was not cached
      * It is possible, but I think unlikely in most cases, that order will matter here. The module-level code may instantiate objects from other reloaded modules. We won't worry about these order specific concerens for now, unless our testing hits this issue. If it is an issue then we may need to store the module loading order (probably as a tree) or more forcefully unload all modules at the start of a reload
4. Before the reload, configure the `Factory` and `Builder` objects to properly register the items required by the end user's application
    * We will start out by brute forcing this. Scan the user project files and search for all of the kv files and py files. Find the widget & App classes within the py files and get their names. 
5. Add two visualization instructions:
    * VisualizeKivyApp: This instruction should accept a class name, along with the module path, and allow the app to be visualized directly within the visualization process. The visualization code will need to import the module containing the class, instantiate the App, and run the App
      * Some App classes may require some special setup code. This is an edge case since all Apps should be able to be modified to execute using the `Run()` method. In theory we could allow the user to specify their own App initialization script - but let's leave that as a possible future enhancement 
    * VisualizeKivyWidget: This instruction will also accept a class name and module path. For now the widget will be simply visualized using the default widget properties. This is a very significant shortcoming. Most widget require some setup to have a meaningful visualization. Adding this feature will be a priority once the hot reload capability has been demonstrated. 




# Implementation Questions


# What information does the visualization subprocess need to execute a user-defined `App` class?

# How can we reset the kivy Factory Object?

# How can we reset the kivy Builder Object?